<a href="https://colab.research.google.com/github/HWAN722/Deep-learning-models/blob/main/Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Self-Attention
Self-Attention is a mechanism that computes the relevance of each element in a sequence with respect to all other elements **in the same sequence**. It is widely used in natural language processing (NLP) to capture dependencies within a sequence.
* **Input**: A single sequence (e.g., a sequence of word embeddings from a sentence).
* **Purpose**: It calculates relationships between elements within the same input sequence to better understand the context.

**Formula**:

For an input sequence $X = [x_1, x_2, ..., x_n]$, Self-Attention is computed as:


$$ \text{Attention}(Q, K, V) = \text{softmax}\left( \frac{QK^T}{\sqrt{d_k}} \right)V $$


where *Q* , *K*, and *V* are the Query, Key, and Value matrices derived from the input *X*.




In [2]:
import torch
import torch.nn as nn

class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert self.head_dim * heads == embed_size, "Embedding size must be divisible by heads"

        self.values = nn.Linear(self.head_dim, embed_size, bias=False)
        self.keys = nn.Linear(self.head_dim, embed_size, bias=False)
        self.queries = nn.Linear(self.head_dim, embed_size, bias=False)
        self.fc_out = nn.Linear(embed_size, embed_size)

    def forward(self, values, keys, query, mask):
        N = query.shape[0]
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        # Split embedding into self.heads different pieces
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        queries = query.reshape(N, query_len, self.heads, self.head_dim)

        energy = torch.matmul(queries.transpose(1, 2), keys.transpose(1, 2).transpose(2, 3))  # (n, h, q, k)
        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))
        attention = torch.softmax(energy / (self.embed_size ** 0.5), dim=3)

        out = torch.matmul(attention, values.transpose(1, 2))  # (n, h, q, d)
        out = out.transpose(1, 2).contiguous().view(N, query_len, self.heads * self.head_dim)  # Reshape

        out = self.fc_out(out)
        return out
